In [9]:
%pip -q install numpy
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [10]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix
    
    def __add__(self, other):
        if len(self.matrix) != len(other.matrix) or len(self.matrix[0]) != len(other.matrix[0]):
            raise ValueError("Матрицы должны быть одинакового размера для сложения")
        
        result = []
        for i in range(len(self.matrix)):
            row = []
            for j in range(len(self.matrix[0])):
                row.append(self.matrix[i][j] + other.matrix[i][j])
            result.append(row)
        
        return Matrix(result)
    
    def __sub__(self, other):
        if len(self.matrix) != len(other.matrix) or len(self.matrix[0]) != len(other.matrix[0]):
            raise ValueError("Матрицы должны быть одинакового размера для вычитания")
        
        result = []
        for i in range(len(self.matrix)):
            row = []
            for j in range(len(self.matrix[0])):
                row.append(self.matrix[i][j] - other.matrix[i][j])
            result.append(row)
        
        return Matrix(result)
    
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] * other)
                result.append(row)
            
            return Matrix(result)
        
        if len(self.matrix[0]) != len(other.matrix):
            raise ValueError("Количество столбцов в первой матрице должно быть равно количеству строк во второй матрице для умножения")
        
        result = []
        for i in range(len(self.matrix)):
            row = []
            for j in range(len(other.matrix[0])):
                sum = 0
                for k in range(len(other.matrix)):
                    sum += self.matrix[i][k] * other.matrix[k][j]
                row.append(sum)
            result.append(row)
        
        return Matrix(result)
    
    def __truediv__(self, other):
        if isinstance(other, (int, float)):
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] / other)
                result.append(row)
            
            return Matrix(result)
        
        raise ValueError("Деление матриц не определено")
    
    def __matmul__(self, other):
        if len(self.matrix[0]) != len(other.matrix):
            raise ValueError("Количество столбцов в первой матрице должно быть равно количеству строк во второй матрице для умножения скалярного произведения")
        
        result = 0
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):
                result += self.matrix[i][j] * other.matrix[j][i]
        
        return result

In [11]:
matrix1 = [[1, 2, 3], 
           [4, 5, 6]]

matrix2 = [[7, 8, 9], 
           [10, 11, 12]]

my_matrix1 = Matrix(matrix1)
my_matrix2 = Matrix(matrix2)

result_matrix = my_matrix1 + my_matrix2
print("Сумма матриц (через Matrix):")
for row in result_matrix.matrix:
    print(row)

np_matrix1 = np.array(matrix1)
np_matrix2 = np.array(matrix2)

np_result_matrix = np_matrix1 + np_matrix2
print("Сумма матриц (через NumPy):")
for row in np_result_matrix:
    print(row)

Сумма матриц (через Matrix):
[8, 10, 12]
[14, 16, 18]
Сумма матриц (через NumPy):
[ 8 10 12]
[14 16 18]


In [12]:
result_matrix = my_matrix1 - my_matrix2
print("Вычитание матриц (через Matrix):")
for row in result_matrix.matrix:
    print(row)

np_result_matrix = np_matrix1 - np_matrix2
print("Вычитание матриц (через NumPy):")
for row in np_result_matrix:
    print(row)

Вычитание матриц (через Matrix):
[-6, -6, -6]
[-6, -6, -6]
Вычитание матриц (через NumPy):
[-6 -6 -6]
[-6 -6 -6]


In [13]:
matrix1 = [[1, 2, 3], 
           [4, 5, 6]]

matrix2 = [[7, 8, 9], 
           [10, 11, 12],
           [3, 5, 7]]

my_matrix1 = Matrix(matrix1)
my_matrix2 = Matrix(matrix2)

result_matrix = my_matrix1 * my_matrix2
print("Умножение матриц (через Matrix):")
for row in result_matrix.matrix:
    print(row)

np_matrix1 = np.array(matrix1)
np_matrix2 = np.array(matrix2)

np_result_matrix = np_matrix1 @ np_matrix2
print("Умножение матриц (через NumPy):")
for row in np_result_matrix:
    print(row)

Умножение матриц (через Matrix):
[36, 45, 54]
[96, 117, 138]
Умножение матриц (через NumPy):
[36 45 54]
[ 96 117 138]


In [14]:
result_matrix = my_matrix1 / 2
print("Деление матрицы на скаляр (через Matrix):")
for row in result_matrix.matrix:
    print(row)

np_result_matrix = np_matrix1 / 2
print("Деление матрицы на скаляр (через NumPy):")
for row in np_result_matrix:
    print(row)

Деление матрицы на скаляр (через Matrix):
[0.5, 1.0, 1.5]
[2.0, 2.5, 3.0]
Деление матрицы на скаляр (через NumPy):
[0.5 1.  1.5]
[2.  2.5 3. ]


In [15]:
result = my_matrix1 @ my_matrix2
print("Скалярное произведение матриц (через Matrix):")
print(result)

np_result = np_matrix1 @ np_matrix2
print("Скалярное произведение матриц (через NumPy):")
print(np_result)

Скалярное произведение матриц (через Matrix):
153
Скалярное произведение матриц (через NumPy):
[[ 36  45  54]
 [ 96 117 138]]
